In [1]:
!pip install Sastrawi
!pip install googletrans

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import googletrans

import time

import warnings
warnings.filterwarnings("ignore")

## Concat the Data

In [6]:
Data_Stream = pd.read_csv('../1_stream/scraping_combined.csv')
Sekunder = pd.read_csv('../1_stream/data_sekunder.csv')
combined_df = pd.concat([Sekunder, Data_Stream], ignore_index=True)
display(combined_df.head())
display(combined_df.tail())

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,Tanggal,Tahun
0,e59cbaf2-3536-4cbe-a60b-1abc7ff37b8a,Hendra,https://play-lh.googleusercontent.com/a-/ALV-U...,Banyak penjual barang yang menjual tidak sesua...,1,0,NaN,2024-02-03 10:16:34,"Hi Toppers, silakan sampaikan kritik/saran ata...",2024-02-03 10:20:07,NaN,NaN,NaN
1,2610c38c-b4fe-4039-a22b-39e1e0142561,Anang Ahmad Mulawarman Samalewa,https://play-lh.googleusercontent.com/a-/ALV-U...,Gk jelas tiba² pembatalan pesanan katanya tida...,1,0,3.251,2024-02-03 10:11:11,"Hi Toppers, silakan sampaikan kritik/saran ata...",2024-02-03 10:20:09,3.251,NaN,NaN
2,96df241d-e7e8-4581-8a7e-476365fa38b2,Mohamad Allwi,https://play-lh.googleusercontent.com/a-/ALV-U...,. Expedisi nya ga jelas.. Kirim paket aja smpa...,1,0,3.251,2024-02-03 10:09:59,"Toppers, mohon maaf atas kendala yang kamu ala...",2024-02-03 10:20:05,3.251,NaN,NaN
3,ab2222f9-b243-47be-9a08-cc799d7a8775,Queensha Amara,https://play-lh.googleusercontent.com/a/ACg8oc...,Parah beli barang malah di tuduh manipulasi da...,1,0,3.250,2024-02-03 10:09:07,"Halo Toppers, terima kasih atas rating yang ka...",2024-02-03 10:20:08,3.250,NaN,NaN
4,63e11d25-48c4-431d-9b66-4e57286cadc1,Sapuani Basri Vlog,https://play-lh.googleusercontent.com/a-/ALV-U...,terimakasih ok bangat,5,0,3.250,2024-02-03 10:05:37,"Toppers, terima kasih atas rating yang kamu be...",2024-02-03 10:20:04,3.250,NaN,NaN


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,Tanggal,Tahun
593366,6fee1fe0-cec3-430a-b573-e71e269f893d,Dewa Gede Yogi Adwesta,https://play-lh.googleusercontent.com/a-/ALV-U...,"stop pake sapx sebagai gratis ongkir oi, baran...",1,1,3.251,2024-02-04 06:52:49,"Halo Toppers, terima kasih atas rating yang ka...",2024-02-04 07:00:02,3.251,2024-02-04 06:52:49,2024.0
593367,32a0b094-9b3f-497a-82a8-57a6dbd81da4,Irvano Budiyanto,https://play-lh.googleusercontent.com/a/ACg8oc...,untuk kurir2 yang katanya rekomendasi tokopedi...,1,3,3.251,2024-02-04 05:42:17,"Halo Toppers, terima kasih atas rating yang ka...",2024-02-04 06:00:02,3.251,2024-02-04 05:42:17,2024.0
593368,748d6bd8-696b-43a9-b1d6-8d769747e11b,Widodo Aditya A.N,https://play-lh.googleusercontent.com/a/ACg8oc...,"Slow respon, no solution 🤗",1,1,3.250,2024-02-04 02:04:35,"Hi Toppers, silakan sampaikan kritik/saran ata...",2024-02-04 02:20:07,3.250,2024-02-04 02:04:35,2024.0
593369,01586f01-b885-4990-8a47-456472163210,chandra “condor99” hermanto,https://play-lh.googleusercontent.com/a-/ALV-U...,Ongkir nya terlalu mahal jauh dibanding oren. ...,3,1,3.250,2024-02-04 00:55:23,"Makasih ratingnya, Toppers. Ke depannya kami a...",2024-02-04 01:00:07,3.250,2024-02-04 00:55:23,2024.0
593370,7b7f9785-dca9-45ef-9526-a48c31f0f2b1,Alex Xander,https://play-lh.googleusercontent.com/a-/ALV-U...,"Dear Tokped, saya sering menggunakan aplikasi ...",3,0,NaN,2024-02-04 00:19:38,"Terima kasih atas rating yang kamu berikan, To...",2024-02-04 00:20:02,NaN,2024-02-04 00:19:38,2024.0


## Data Cleaning

In [7]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593371 entries, 0 to 593370
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   reviewId              593371 non-null  object 
 1   userName              593369 non-null  object 
 2   userImage             593371 non-null  object 
 3   content               593370 non-null  object 
 4   score                 593371 non-null  int64  
 5   thumbsUpCount         593371 non-null  int64  
 6   reviewCreatedVersion  439716 non-null  object 
 7   at                    593371 non-null  object 
 8   replyContent          240532 non-null  object 
 9   repliedAt             240532 non-null  object 
 10  appVersion            439716 non-null  object 
 11  Tanggal               13286 non-null   object 
 12  Tahun                 13286 non-null   float64
dtypes: float64(1), int64(2), object(10)
memory usage: 58.9+ MB


### Check nan value

In [8]:
combined_df.isnull().sum()

reviewId                     0
userName                     2
userImage                    0
content                      1
score                        0
thumbsUpCount                0
reviewCreatedVersion    153655
at                           0
replyContent            352839
repliedAt               352839
appVersion              153655
Tanggal                 580085
Tahun                   580085
dtype: int64

### Check Duplicate

In [9]:
combined_df.duplicated().sum()
print("There are", combined_df.duplicated().sum(), "duplicate rows in the dataset.")

There are 0 duplicate rows in the dataset.


In [10]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593371 entries, 0 to 593370
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   reviewId              593371 non-null  object 
 1   userName              593369 non-null  object 
 2   userImage             593371 non-null  object 
 3   content               593370 non-null  object 
 4   score                 593371 non-null  int64  
 5   thumbsUpCount         593371 non-null  int64  
 6   reviewCreatedVersion  439716 non-null  object 
 7   at                    593371 non-null  object 
 8   replyContent          240532 non-null  object 
 9   repliedAt             240532 non-null  object 
 10  appVersion            439716 non-null  object 
 11  Tanggal               13286 non-null   object 
 12  Tahun                 13286 non-null   float64
dtypes: float64(1), int64(2), object(10)
memory usage: 58.9+ MB


## DONWLOAD DATA

In [11]:
# ======================
# 4) Export ke CSV
# ======================
output_file = "hasil_cleaning.csv"
combined_df.to_csv(output_file, index=False, encoding="utf-8")
print(f"\n✅ File berhasil disimpan sebagai: {output_file}")


✅ File berhasil disimpan sebagai: hasil_cleaning.csv
